<a href="https://colab.research.google.com/github/pull-ups/personality-type-recommendation/blob/recommendation/movie-summary%26music-lyric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://sec

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.l

In [4]:
import numpy as np
import pandas as pd
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
import re

from gensim.models.word2vec import Word2Vec
import gensim
import urllib.request

from google.colab import drive
drive.mount('/content/gdrive')

okt = Okt()
kkma=Kkma()


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  6087k      0 --:--:-- --:--:-- --:--:-- 24.9M
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

Mounted at /content/gdrive


In [5]:
def tokenizer(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return(o)


In [6]:
#legacy-----
def get_score_with_weight(answer_centered, i, word):
  score=0
  if (i==0 or i==1):
    if answer_centered[i]<0:
      score=np.abs(answer_centered[i]) * model.wv.similarity('잔잔', word)
    else:
      score=np.abs(answer_centered[i]) * model.wv.similarity('활발', word)

  if (i==2 or i==3):
    if answer_centered[i]<0:
      score=np.abs(answer_centered[i]) * model.wv.similarity('이성', word)
    else:
      score=np.abs(answer_centered[i]) * model.wv.similarity('본능', word)

  if (i==4 or i==5):
    if answer_centered[i]<0:
      score=np.abs(answer_centered[i]) * model.wv.similarity('객관', word)
    else:
      score=np.abs(answer_centered[i]) * model.wv.similarity('주관', word)

  if (i==6):
    if answer_centered[i]<0:
      score=np.abs(answer_centered[i]) * model.wv.similarity('행복', word)
    else:
      score=np.abs(answer_centered[i]) * model.wv.similarity('위로', word)

  if (i==7):
    if answer_centered[i]<0:
      score=np.abs(answer_centered[i]) * model.wv.similarity('사랑', word)
    else:
      score=np.abs(answer_centered[i]) * model.wv.similarity('홀로', word)
  return score

In [7]:
import gensim
model = gensim.models.Word2Vec.load('/content/gdrive/MyDrive/Colab Notebooks/ko.bin')

In [12]:
movie=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/personality based recommendation/movie_sample.csv', encoding='cp949')
stopwords = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/personality based recommendation/한국어불용어.txt", sep='\t', encoding='utf-8')['형태'].to_list()


In [13]:
summary=movie['summary']
summary

0     1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...
1     나를 세상에 태어나게 한 부모님을 고소하고 싶어요...출생기록조차 없이 살아온 어쩌...
2     컴퓨터 모니터를 통해 세상 밖으로 나온 의문의 알.타이치와 히카리 남매 앞에 디지몬...
3     귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다.하지만 눈을 떠보니 ...
4     누구보다 위트 있고 호기심 많은 매력 부자 어기제이콥 트렘블레이.하지만 남들과 다른...
                            ...                        
95    1910년의 런던. 중절모를 눌러쓴 신사들과 우아한 나들이 복 차림의 귀부인들 사이...
96    8비트 게임 다고쳐 펠릭스에서 건물을 부수는 악당 주먹왕 랄프. 30년째 매일같이 ...
97    어린 시절 약물 중독에 걸린 엄마와 강제로 헤어진 후 여러 가정을 전전하며 커가던 ...
98    어렸을 때부터 함께 경극을 해온 두지장국영와 시투장풍의. 세상에 둘도 없는 절친한 ...
99    넉넉하지는 않지만 누구보다 행복한 가장 리키안정적인 생활을 꿈꾸며 택배 회사에 취직...
Name: summary, Length: 100, dtype: object

In [14]:
texts=[]
for doc in summary:
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    try:
        word_token = kkma.morphs(hangul.sub('', str(doc)))

        stopped_tokens = [word for word in word_token if not word in stopwords]
        stopped_tokens = [word for word in stopped_tokens if len(word)>1]
        texts.append(stopped_tokens)
        
    except:
        texts.append('')

In [115]:
MBTI[0]

['활발', '유쾌', '씩씩', '자유']

In [15]:
E_word = ['활발', '유쾌', '씩씩', '자유' ]
I_word = ['조용',  '잔잔', '신중', '깊이']
S_word = ['오감', '감각', '경험', '정확', '철저']
N_word = ['영감', '신속', '일', '추구']
F_word = ['사람', '관계', '설명']
T_word = ['진실', '사실', '논리', '분석']
P_word = ['변화', '자율', '융통성']
J_word = ['분명', '체계', '철저']
MBTI=[E_word, I_word, S_word, N_word, F_word, T_word, P_word, J_word]

def get_score_v2(answer_vector, i, word):
  score=0
  for compare_word in MBTI[i]:
    score+=answer_vector[i] * model.wv.similarity(compare_word, word)
  score/=len(MBTI[i])  
  return score

In [16]:
def recommend_movie(answer):
  print(answer)
  answer=np.array(answer)
  idx=0
  scores=[]
  for sentence in movie['summary']:
    word_tokenized=okt.morphs(sentence)
    score=0
    not_in_voca=0
    for word in word_tokenized:
      try:
        for i in range(len(answer)):
          score+=get_score_v2(answer, i, word)
            
      except Exception as e:
        not_in_voca+=1
    score/=(len(word_tokenized)-not_in_voca)
    scores.append(score)
    idx+=1
    
  np.array(scores).argsort()[-10:][::-1]
  for i in np.array(scores).argsort()[-10:][::-1]:
    print(movie.loc[i, 'title'])
    print(movie.loc[i, 'summary'])
    print(movie.loc[i, 'genre'])
    print('---------------')   

In [17]:
recommend_movie([7,1,1,1,1,1,1,1])

[7, 1, 1, 1, 1, 1, 1, 1]
미안해요, 리키
넉넉하지는 않지만 누구보다 행복한 가장 리키안정적인 생활을 꿈꾸며 택배 회사에 취직하지만생각과는 다른 일상이 전개되고 화목했던 가족은 뜻밖의 난관에 부딪히는데.. 성실하게 행복을 찾고 싶었던 리키의 우리의 이야기가 찾아온다.
드라마
---------------
극장판 바이올렛 에버가든
친애하는 길베르트 소령님오늘도 또 당신을 떠올리고 말았습니다. 무엇을 보든 무엇을 하든 당신이 떠오릅니다. 시간이 지나도 당신과 보냈던 기억은 선명하게 되살아납니다. 당신은 날 곁에 두었고 아무것도 모르는 제게 살아가는 방법을 가르쳐 주셨고처음으로 사랑이라는 감정을 알려주셨습니다. 그래서 이렇게 또 편지를 쓰게 됩니다. 언젠가 이 편지가 당신에게 닿기를 바라며
애니메이션,드라마,판타지
---------------
가나의 혼인잔치 언약
우리는 소망을 잃어버렸다.예수의 언약을 잊어버렸다.하지만 최근 연구를 통해숨겨둔 언약의 비밀이 드러나게 되는데예수는 무엇을 약속하셨는가역사를 통해 드러난 충격적인 비밀이우리의 영성을 두드린다.가장 안전하고 정확하게 언약 속 비밀을 맞이하라
다큐멘터리,드라마
---------------
주전장
일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인 유튜버 미키 데자키. 그는 일본군위안부에 관한 기사를 쓴 기자가 우익들에게 인신공격 당하는 것을 보며 왜 그토록 이들이 이 문제를 감추려고 하는지 궁금해졌다.호기심을 안고 찾아간 그들은 무시무시한 이야기를 전하고 그들의 주장을 반격하는 또 다른 인물들을 만나면서 숨겨진 비밀을 발견하게 되는데 숨 쉴 틈 없는 전쟁이 시작된다
다큐멘터리
---------------
아마데우스
천재를 시기한 평범한 궁정음악가의 질투에서 시작된 광기에 찬 파멸의 서곡비엔나 왕실의 궁정음악가 살리에리F.머레이 아브라함는 새롭게 떠오르는 천재 작곡가 모짜르트톰 헐스의 소문을 듣고 그의 천재성을 확인하려 한다. 하지만 방탕하고 오만한 모짜르트의 행

In [ ]:
recommend_movie([1,1,1,1,1,7,1,1])

In [ ]:
recommend_movie([1,1,7,1,1,1,1,1])

In [90]:
music=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/personality based recommendation/Bugs_lyric_sample.csv', sep=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [91]:
music['Lyrics']

0       Move a step closer  You know that I want you  ...
1       You were looking at me  Like you wanted to sta...
2       One two three  I been running from the pain Tr...
3       Trying to forget you  I'm all alone  In a ghos...
4       Someday we'll be a million miles away  Waking ...
                              ...                        
3281    I saw stars on the pavement  California dreams...
3282    Call me what you wanna  I'll be what you wanna...
3283    You wanna see if I'm down Turn it up loud, zoo...
3284    I can see the future, that ain't much between ...
3285     Last night, I had a dream  Was up in the clou...
Name: Lyrics, Length: 3286, dtype: object

In [92]:
eng_lyric_list=[]
for i in range(music.shape[0]):
  text=music['Lyrics'][i]
  if not is_hangul(text):
    eng_lyric_list.append(i)


len(eng_lyric_list)/music.shape[0]

0.2845404747413268

In [93]:
music.drop(music.index[eng_lyric_list], inplace=True)
music.reset_index(inplace=True)
music.drop(['index', 'Unnamed: 0'],axis=1, inplace=True)

In [94]:
music

,CategoryName,ThemeName,ThemeUrl,Title,Artist,Album,Lyrics
0,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,FINE NEW DAY (Feat. TAK),애런(ARRAN)CONNECT,PUZZLE 9 PIECES,Be fine things will be fine Be mild turn down...
1,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,굿띵 (good thing) (Feat. WONDERLUST),로즈(Rose),굿띵 (good thing),I loving driving 널 데려가는 이길 퇴근길이라서 기분 좋아 널 옆...
2,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,Aqua Man,빈지노(Beenzino),2 4 : 2 6,어항 속에 갇힌 고기들보다 어쩌면 내가 좀 더 멍청할지 몰라 너가 먹이처럼 던진...
3,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,Blue Lemonade,Red Velvet (레드벨벳),Summer Magic - Summer Mini Album,(Pop pop pop) ah ah ah ah (Pop pop pop) ah ah...
4,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,BLUE MOON (Prod. GroovyRoom),"효린CONNECT , 창모(CHANGMO)",BLUE MOON,어서 날 알아보길 바래 나 없는 넌 안 되겠다고 말해 깨고 또 깨어나도 반복되는...
...,...,...,...,...,...,...,...
2346,뮤직스타일,일렉트로닉,https://music.bugs.co.kr/musicpd?tag_id=1036,푹,109,깊은 밤 너와 나,푹 깊이 삼켰던 숨을 내쉬어 아무도 뭐라 하지 않아 슥 하고 스쳐간 시간들 중 ...
2347,뮤직스타일,일렉트로닉,https://music.bugs.co.kr/musicpd?tag_id=1036,쉬어가도 돼요,디에이드(The Ade),사랑론,말하지 않아도 알아요 길어진 숨 끝이 말하죠 한숨에 담긴 그 고민들 음 괜찮아...
2348,뮤직스타일,일렉트로닉,https://music.bugs.co.kr/musicpd?tag_id=1036,최선,단칸방 로맨스,최선,밤새 나누었던 긴 통화는 이제 의무감으로 하네요 사랑 한다고 난 말했어요 사랑...
2349,뮤직스타일,일렉트로닉,https://music.bugs.co.kr/musicpd?tag_id=1036,I Go,Peggy Gou(페기 구)CONNECT,I Go,삶에 지친 내 모습 또 비쳐 지네 사람들이 네게 행복 하냐 물어 보네 힘든 내맘...


In [108]:
music.drop_duplicates(['Title'], inplace=True)
music.dropna(axis=0, inplace=True)
music.reset_index(inplace=True)
music.drop(['index'], axis=1, inplace=True)

In [109]:
music

,CategoryName,ThemeName,ThemeUrl,Title,Artist,Album,Lyrics
0,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,FINE NEW DAY (Feat. TAK),애런(ARRAN)CONNECT,PUZZLE 9 PIECES,Be fine things will be fine Be mild turn down...
1,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,굿띵 (good thing) (Feat. WONDERLUST),로즈(Rose),굿띵 (good thing),I loving driving 널 데려가는 이길 퇴근길이라서 기분 좋아 널 옆...
2,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,Aqua Man,빈지노(Beenzino),2 4 : 2 6,어항 속에 갇힌 고기들보다 어쩌면 내가 좀 더 멍청할지 몰라 너가 먹이처럼 던진...
3,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,Blue Lemonade,Red Velvet (레드벨벳),Summer Magic - Summer Mini Album,(Pop pop pop) ah ah ah ah (Pop pop pop) ah ah...
4,뮤직스타일,신나는,https://music.bugs.co.kr/musicpd?tag_id=429,BLUE MOON (Prod. GroovyRoom),"효린CONNECT , 창모(CHANGMO)",BLUE MOON,어서 날 알아보길 바래 나 없는 넌 안 되겠다고 말해 깨고 또 깨어나도 반복되는...
...,...,...,...,...,...,...,...
518,뮤직스타일,어쿠스틱한,https://music.bugs.co.kr/musicpd?tag_id=8010,소년,홍이삭,놓치고 싶지 않은 사소한 것들,여기저기서 단풍잎 같은 슬픈 가을이 뚝뚝 떨어진다 단풍잎 떨어져 나온 자리마다 ...
519,뮤직스타일,어쿠스틱한,https://music.bugs.co.kr/musicpd?tag_id=8010,푹,109,깊은 밤 너와 나,푹 깊이 삼켰던 숨을 내쉬어 아무도 뭐라 하지 않아 슥 하고 스쳐간 시간들 중 ...
520,뮤직스타일,어쿠스틱한,https://music.bugs.co.kr/musicpd?tag_id=8010,쉬어가도 돼요,디에이드(The Ade),사랑론,말하지 않아도 알아요 길어진 숨 끝이 말하죠 한숨에 담긴 그 고민들 음 괜찮아...
521,뮤직스타일,어쿠스틱한,https://music.bugs.co.kr/musicpd?tag_id=8010,최선,단칸방 로맨스,최선,밤새 나누었던 긴 통화는 이제 의무감으로 하네요 사랑 한다고 난 말했어요 사랑...


In [110]:
texts=[]
for doc in music['Lyrics']:
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    try:
        word_token = kkma.morphs(hangul.sub('', str(doc)))

        stopped_tokens = [word for word in word_token if not word in stopwords]
        stopped_tokens = [word for word in stopped_tokens if len(word)>1]
        texts.append(stopped_tokens)
        
    except:
        texts.append('')

In [111]:
texts[0]

['오늘',
 '다르',
 '매일',
 '지나오',
 '골목길',
 '심심',
 '낙서',
 '이상',
 '느낌',
 '안되',
 '더라',
 '대충',
 '에는',
 '그리하',
 '여도',
 '오늘',
 '너와',
 '달리',
 '내가',
 '래왔',
 '여태',
 '차분',
 '눌르',
 '기분',
 '기분',
 '내일',
 '아침',
 '돌아오',
 '까지',
 '기지',
 '내놓',
 '그때',
 '아무',
 '해보',
 '오늘',
 '아무',
 '나란히',
 '어서',
 '떨어지',
 '같이',
 '별거',
 '하늘',
 '느낌',
 '안되',
 '더라',
 '대충',
 '에는',
 '그리하',
 '여도',
 '오늘',
 '너와',
 '달리',
 '내가',
 '래왔',
 '여태',
 '차분',
 '눌르',
 '기분',
 '기분',
 '내일',
 '아침',
 '돌아오',
 '까지',
 '기지',
 '내놓',
 '그때',
 '아무',
 '해보',
 '그때',
 '아무',
 '해보',
 '어야']

In [112]:
def recommend_music(answer):
  print(answer)
  answer=np.array(answer)
  idx=0
  scores=[]
  for sentence in music['Lyrics'][:100]:
    word_tokenized=okt.morphs(sentence)
    score=0
    not_in_voca=0
    for word in word_tokenized:
      try:
        for i in range(len(answer)):
          score+=get_score_v2(answer, i, word)
            
      except Exception as e:
        not_in_voca+=1
    score/=(len(word_tokenized)-not_in_voca)
    scores.append(score)
    idx+=1
    
  np.array(scores).argsort()[-10:][::-1]
  for i in np.array(scores).argsort()[-10:][::-1]:
    print(i)
    print(music.loc[i, 'Title'])
    print(music.loc[i, 'Lyrics'])
    print(music.loc[i, 'ThemeName'])
    print('---------------')   

In [113]:
recommend_music([1,7,1,1,1,1,1,1])

[1, 7, 1, 1, 1, 1, 1, 1]
19
SUMMERIDE
몸이 지쳤을 때 밖을 바라봐  자연과 함께 하나가 돼버리네  설레이는 마음 내가 달려가  위치 보내면 oh oh  Where u at? Pick u up  올라타 딱이네  We can go anywhere  Cruising everywhere  날씨도 최고네  분위기 거의 LA  We can go anywhere  Cruising everywhere  Summertime  We can play  고민 no 자유롭게  Girl open your window  Summertime  We can play  고민 no 자유롭게  Girl open your window  Summertime yeah SUMMERIDE  Summertime yeah SUMMERIDE  Summertime yeah SUMMERIDE  Steady in my Chevy  Summertime yeah SUMMERIDE  Summertime yeah SUMMERIDE  Summertime yeah SUMMERIDE  Steady in my Chevy  오랜만에 만난 여름날  Trailblazing through the streets  즐거운 이 느낌 엄청나  이 추억 간직하고파  이따 집에 가서 오늘 날짜  메모지에 적어놔  기분 super up  I don't give a what  We be drivin' drivin' girl  I can do this all day  어느 때도 언제나  Baby girl u can call me  매일 vacation  함께 있으면  도로도 천국이야  이 순간 맛있어 girl  I can taste it  Never stop green light  Always giddy up, let’s go  Summertime  We can play  고민 no 자유롭게  Girl open your window  Summertime  We can play  고민 no 자유롭게  Girl open yo

In [114]:
recommend_music([7,1,1,1,1,1,1,1])

[7, 1, 1, 1, 1, 1, 1, 1]
43
Let's!
Let's sit in my car  무작정 떠나 네 주위를 봐  놀라지는 마 길고 긴 시간을 달려서  저기 바다가 보여  새로운 풍경과 사람들  이제 나는 떠날래  Hey look at the stars  별이 빛나는 밤 하늘 좀 봐  우린 뭘 놓치고 살았는지  좋아하고 미워하는 건 다 지쳤어  오늘은 다 잊고 즐기자  아무도 우릴 모르니까  Let's sit in my car get out of this town  Let's sit in my car get out of this town  All day baby all night long  우리 노래 부르자  All day baby all night long  우리 꿈을 꿔보자  Let's jump in my car  모래성을 짓고 무알콜 한잔  취할 것만 같아 기분 좋은 밤 기분 좋은 Vibe  이 음악 제목이 뭐지 상관없어 춤추면 되지  Let's sit in my car get out of this town  Let's sit in my car get out of this town  All day baby all night long  우리 노래 부르자  All day baby all night long  우리 꿈을 꿔보자  Let's sit in my car get out of this town  Let's sit in my car get out of this town  All day baby all night long  우리 노래 부르자  All day baby all night long  우리 꿈을 꿔보자  All day all night long  함께 노래 부르자  Say all day all night long  함께 꿈을 꿔보자  Say all day all night long  함께 노래 부르자  Say all day all night long  우리 춤을 춰보자  Let's sit in my car get out of this

In [60]:
movie['summary']

0     1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...
1     나를 세상에 태어나게 한 부모님을 고소하고 싶어요...출생기록조차 없이 살아온 어쩌...
2     컴퓨터 모니터를 통해 세상 밖으로 나온 의문의 알.타이치와 히카리 남매 앞에 디지몬...
3     귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다.하지만 눈을 떠보니 ...
4     누구보다 위트 있고 호기심 많은 매력 부자 어기제이콥 트렘블레이.하지만 남들과 다른...
                            ...                        
95    1910년의 런던. 중절모를 눌러쓴 신사들과 우아한 나들이 복 차림의 귀부인들 사이...
96    8비트 게임 다고쳐 펠릭스에서 건물을 부수는 악당 주먹왕 랄프. 30년째 매일같이 ...
97    어린 시절 약물 중독에 걸린 엄마와 강제로 헤어진 후 여러 가정을 전전하며 커가던 ...
98    어렸을 때부터 함께 경극을 해온 두지장국영와 시투장풍의. 세상에 둘도 없는 절친한 ...
99    넉넉하지는 않지만 누구보다 행복한 가장 리키안정적인 생활을 꿈꾸며 택배 회사에 취직...
Name: summary, Length: 100, dtype: object

In [68]:
music['Lyrics'][:10]

0    Be fine things will be fine  Be mild turn down...
1    I loving driving  널 데려가는 이길  퇴근길이라서 기분 좋아  널 옆...
2    어항 속에 갇힌 고기들보다  어쩌면 내가 좀 더 멍청할지 몰라  너가 먹이처럼 던진...
3    (Pop pop pop) ah ah ah ah  (Pop pop pop) ah ah...
4    어서 날 알아보길 바래  나 없는 넌 안 되겠다고 말해  깨고 또 깨어나도 반복되는...
5    Where to go   Where to go  Where to go   Fly a...
6    Your love smells good,   wish I could get it o...
7    와요 와요 와요 와    너와 나 지금부터 잠시 12345  눈을 감고 가만히 서 ...
8    거기선 모두가 노랠 하고 산대요  부서지는 파도 앞에 살면서  가장 낯설은 도시, ...
9    귀여워 귀여워 웃을때 귀여워  너무 귀여운 나만의 연인 그대  멋있어 멋있어 너무 ...
Name: Lyrics, dtype: object

In [116]:
word_tokenized=okt.morphs(music['Lyrics'][0])

In [117]:
word_tokenized

['Be',
 'fine',
 'things',
 'will',
 'be',
 'fine',
 'Be',
 'mild',
 'turn',
 'down',
 'lay',
 'off',
 'leave',
 'it',
 'on',
 'flow',
 'Be',
 'fine',
 'things',
 'will',
 'be',
 'fine',
 'Be',
 'mild',
 'turn',
 'down',
 'lay',
 'off',
 'leave',
 'it',
 'on',
 'flow',
 '오늘',
 '은',
 '좀',
 '다른',
 '것',
 '같아',
 '매일',
 '지나',
 '왔던',
 '이',
 '좁은',
 '골목길',
 '도',
 '심심했던',
 '저',
 '벽',
 '의',
 '낙서',
 '도',
 '이상하게',
 '느낌',
 '있게',
 '보이',
 '네',
 '꼭',
 '이런',
 '날',
 '은',
 '잘',
 '안되더라',
 'Oh',
 '대충',
 '할',
 '땐',
 '그렇게',
 '잘',
 '되던',
 'Make',
 'up',
 'hair',
 'style',
 '그래도',
 '오늘',
 '은',
 'Ok',
 '너',
 '와',
 '달리',
 '면',
 '돼',
 '그게',
 '내',
 '가',
 '바',
 '래',
 '왔던',
 '거야',
 '여태',
 '차분하게',
 '꾹',
 '눌러',
 '담아',
 '왔던',
 '이',
 '기분',
 '을',
 'Oh',
 '이',
 '기분',
 '을',
 '내일',
 '저',
 '아침',
 '이',
 '돌아오기',
 '전',
 '까지',
 '숨기',
 '지',
 '말고',
 '다',
 '꺼내놓고',
 '나',
 '만',
 '보는',
 '거야',
 '나와',
 '손',
 '을',
 '잡는',
 '거야',
 'Babe',
 '그때',
 '까지는',
 '아무',
 '생각',
 '도',
 '안',
 '해보는',
 '거야',
 'Ay',
 'Get',
 'ready',
 'for',
 'a',
 'fine'